In [1]:
import pandas as pd
import math
from unidecode import unidecode
import numpy as np
import scipy.io
from scipy.optimize import minimize
from scipy.stats import norm
import matplotlib.pyplot as plt
import os


from scipy.stats import linregress

PATH_UPV = "./DatosExperimento1/UPV"
PATH_UGR = "./DatosExperimento1/UGR"
PATH_VARIABLES = "./variables"

if not os.path.exists('variables'):
    os.makedirs('variables') 

# Leer el archivo Excel en un DataFrame de pandas
df_upv = pd.read_excel(PATH_UPV+'/Exp1_Observadores_UPV.xlsx')
df_ugr = pd.read_excel(PATH_UGR+'/Exp1_Observadores_UGR.xlsx')
    
if not os.path.exists('Resultados_UPV'):
    os.makedirs('Resultados_UPV')    

RESULTS_PATH_UPV = 'Resultados_UPV/0-PROMEDIOS'
if not os.path.exists(RESULTS_PATH_UPV):
    os.makedirs(RESULTS_PATH_UPV)

if not os.path.exists('Resultados_UGR'):
    os.makedirs('Resultados_UGR')
    
RESULTS_PATH_UGR = 'Resultados_UGR/0-PROMEDIOS'
if not os.path.exists(RESULTS_PATH_UGR):
    os.makedirs(RESULTS_PATH_UGR)
    
if not os.path.exists('Resultados_TOTAL'):
    os.makedirs('Resultados_TOTAL')
    
RESULTS_PATH_TOTAL = 'Resultados_TOTAL'

# Array de ficheros .mat, nos da igual el volunatrio
ficheros_upv = []
ficheros_ugr = []
ficheros_total = []

# Iterar sobre cada fila del DataFrame
for idx, row in df_upv.iterrows():
    ficheros_upv.append(row['Fichero'])
    ficheros_total.append(PATH_UPV+"/"+row['Fichero'])
    
for idx, row in df_ugr.iterrows():
    ficheros_ugr.append(row['Fichero'])
    ficheros_total.append(PATH_UGR+"/"+row['Fichero'])

In [2]:
def normcdf2(x, mu, sigma):
    '''
    Utilizo la función norm.cdf de la librería scipy.stats para calcular la función de distribución acumulada (CDF) 
    de la distribución normal, y luego dividimos el resultado por dos y sumamos 0.5 para ajustar el rango a [0,1].
    '''
    y = (norm.cdf(x, mu, sigma) + 1) / 2
    return y

def normcdf2_minimizer(X, params):
    # X incluye mu y sigma
    mu = X[0]
    sigma = X[1]

    # x/y valores para ajustar
    x = params['x']
    y = params['y']

    # los valores ajustados con mu y sigma dados
    yfit = normcdf2(x, mu, sigma)

    # cálculo del error
    err = np.sum((yfit - y)**2)

    return err

In [3]:
# Variables generales
ruidos = scipy.io.loadmat('ruidosExp1.mat')
ruidos = np.array(ruidos['ruidos'])
minX = np.min(ruidos[:, 2])
maxX = np.max(ruidos[:, 2])
LoopNumber = 16
numberObservations = 10

In [4]:
# UPV
mean_result_upv = None
mean_std_upv = []
for fichero in ficheros_upv:
    data_file = PATH_UPV+'/'+fichero
    result = scipy.io.loadmat(data_file)
    result = np.array(result['Results'])
    if mean_result_upv is None:
        mean_result_upv = result
    else:
        mean_result_upv = mean_result_upv + result 
        # Suma de los resultados
        # Promediar la probabilidad y Promediar el tiempo
mean_result_upv = mean_result_upv/len(ficheros_upv) # Media de los resultados para todos los ficheros de la UPV
    

for i in range(LoopNumber):
    x = ruidos[i*numberObservations:(i+1)*numberObservations, 2]
    ruido_base = ruidos[i*numberObservations:(i+1)*numberObservations, 0][0]
    y = mean_result_upv[i*numberObservations:(i+1)*numberObservations, 0]

    # Valores iniciales de la minimización
    X0 = [np.mean(x), 0.1*np.mean(x)]

    # parámetros para el ajuste
    params = {'x': x, 'y': y}

    # minimización
    ParamsOpt = minimize(normcdf2_minimizer, X0, args=params, method='Nelder-Mead').x

    # media y la desviación estándar de la distribución ajustada
    # Se corresponde al umbral, ruido correspondiente a una prob de detección de 0.75
    meanAndStdDistrib = [ParamsOpt[0], ParamsOpt[1]]
    
    mean_std_upv.append((ruido_base, round(ParamsOpt[0], 6), round(ParamsOpt[1], 6)))
    np.save(RESULTS_PATH_UPV+'/UPV-MeanStdDistribution-' + str(i+1) + '.npy', meanAndStdDistrib)

In [5]:
data_upv = [row for row in [mean_std_upv[i:i+4] for i in range(0, 16, 4)]]
for row in data_upv:
    print(row)

np.save(PATH_VARIABLES+'/data_upv.npy', data_upv)
np.save(PATH_VARIABLES+'/mean_std_upv.npy', mean_std_upv)
mean_std_upv

[(0.01, 0.011607, 0.002403), (0.02, 0.025164, 0.005649), (0.03, 0.039522, 0.009064), (0.04, 0.053994, 0.011909)]
[(0.01, 0.012196, 0.00296), (0.02, 0.02368, 0.004724), (0.03, 0.036786, 0.008197), (0.04, 0.049028, 0.007125)]
[(0.01, 0.012747, 0.002871), (0.02, 0.022552, 0.005572), (0.03, 0.032933, 0.01064), (0.04, 0.043185, 0.015927)]
[(0.01, 0.014579, 0.001328), (0.02, 0.027725, 0.008075), (0.03, 0.037835, 0.009188), (0.04, 0.046935, 0.011525)]


[(0.01, 0.011607, 0.002403),
 (0.02, 0.025164, 0.005649),
 (0.03, 0.039522, 0.009064),
 (0.04, 0.053994, 0.011909),
 (0.01, 0.012196, 0.00296),
 (0.02, 0.02368, 0.004724),
 (0.03, 0.036786, 0.008197),
 (0.04, 0.049028, 0.007125),
 (0.01, 0.012747, 0.002871),
 (0.02, 0.022552, 0.005572),
 (0.03, 0.032933, 0.01064),
 (0.04, 0.043185, 0.015927),
 (0.01, 0.014579, 0.001328),
 (0.02, 0.027725, 0.008075),
 (0.03, 0.037835, 0.009188),
 (0.04, 0.046935, 0.011525)]

In [6]:
# UGR
mean_result_ugr = None
mean_std_ugr = []
for fichero in ficheros_ugr:
    data_file = PATH_UGR+'/'+fichero
    result = scipy.io.loadmat(data_file)
    result = np.array(result['Results'])
    if mean_result_ugr is None:
        mean_result_ugr = result
    else:
        mean_result_ugr = mean_result_ugr + result 
        # Suma de los resultados
        # Promediar la probabilidad y Promediar el tiempo
mean_result_ugr = mean_result_ugr/len(ficheros_ugr) # Media de los resultados para todos los ficheros de la UGR
    

for i in range(LoopNumber):
    x = ruidos[i*numberObservations:(i+1)*numberObservations, 2]
    ruido_base = ruidos[i*numberObservations:(i+1)*numberObservations, 0][0]
    y = mean_result_ugr[i*numberObservations:(i+1)*numberObservations, 0]

    # Valores iniciales de la minimización
    X0 = [np.mean(x), 0.1*np.mean(x)]

    # parámetros para el ajuste
    params = {'x': x, 'y': y}

    # minimización
    ParamsOpt = minimize(normcdf2_minimizer, X0, args=params, method='Nelder-Mead').x

    # media y la desviación estándar de la distribución ajustada
    meanAndStdDistrib = [ParamsOpt[0], ParamsOpt[1]]
    
    mean_std_ugr.append((ruido_base, round(ParamsOpt[0], 6), round(ParamsOpt[1], 6)))
    np.save(RESULTS_PATH_UGR+'/UGR-MeanStdDistribution-' + str(i+1) + '.npy', meanAndStdDistrib)

In [7]:
data_ugr = [row for row in [mean_std_ugr[i:i+4] for i in range(0, 16, 4)]]
for row in data_ugr:
    print(row)
    
np.save(PATH_VARIABLES+'/data_ugr.npy', data_ugr)
np.save(PATH_VARIABLES+'/mean_std_ugr.npy', mean_std_ugr)
mean_std_ugr

[(0.01, 0.010867, 0.001107), (0.02, 0.022187, 0.003187), (0.03, 0.034682, 0.006838), (0.04, 0.047834, 0.007186)]
[(0.01, 0.011123, 0.001766), (0.02, 0.022106, 0.002299), (0.03, 0.032921, 0.004147), (0.04, 0.043683, 0.004548)]
[(0.01, 0.011551, 0.002026), (0.02, 0.020899, 0.003275), (0.03, 0.030758, 0.004312), (0.04, 0.040668, 0.006249)]
[(0.01, 0.01439, 0.00196), (0.02, 0.024073, 0.002754), (0.03, 0.03334, 0.004492), (0.04, 0.043154, 0.006503)]


[(0.01, 0.010867, 0.001107),
 (0.02, 0.022187, 0.003187),
 (0.03, 0.034682, 0.006838),
 (0.04, 0.047834, 0.007186),
 (0.01, 0.011123, 0.001766),
 (0.02, 0.022106, 0.002299),
 (0.03, 0.032921, 0.004147),
 (0.04, 0.043683, 0.004548),
 (0.01, 0.011551, 0.002026),
 (0.02, 0.020899, 0.003275),
 (0.03, 0.030758, 0.004312),
 (0.04, 0.040668, 0.006249),
 (0.01, 0.01439, 0.00196),
 (0.02, 0.024073, 0.002754),
 (0.03, 0.03334, 0.004492),
 (0.04, 0.043154, 0.006503)]

In [8]:
# TOTAL UGR + UPV
mean_result_total = None
mean_std_total = []
for fichero in ficheros_total:
    data_file = fichero
    result = scipy.io.loadmat(data_file)
    result = np.array(result['Results'])
    if mean_result_total is None:
        mean_result_total = result
    else:
        mean_result_total = mean_result_total + result 
        # Suma de los resultados
        # Promediar la probabilidad y Promediar el tiempo
mean_result_total = mean_result_total/len(ficheros_total) # Media de los resultados para todos los ficheros de la TOTAL
    

for i in range(LoopNumber):
    x = ruidos[i*numberObservations:(i+1)*numberObservations, 2]
    ruido_base = ruidos[i*numberObservations:(i+1)*numberObservations, 0][0]
    y = mean_result_total[i*numberObservations:(i+1)*numberObservations, 0]

    # Valores iniciales de la minimización
    X0 = [np.mean(x), 0.1*np.mean(x)]

    # parámetros para el ajuste
    params = {'x': x, 'y': y}

    # minimización
    ParamsOpt = minimize(normcdf2_minimizer, X0, args=params, method='Nelder-Mead').x

    # media y la desviación estándar de la distribución ajustada
    meanAndStdDistrib = [ParamsOpt[0], ParamsOpt[1]]

    mean_std_total.append((ruido_base, round(ParamsOpt[0], 6), round(ParamsOpt[1], 6)))
    np.save(RESULTS_PATH_TOTAL+'/TOTAL-MeanStdDistribution-' + str(i+1) + '.npy', meanAndStdDistrib)

In [9]:
data_total = [row for row in [mean_std_total[i:i+4] for i in range(0, 16, 4)]]
for row in data_total:
    print(row)
    
np.save(PATH_VARIABLES+'/data_total.npy', data_total)
np.save(PATH_VARIABLES+'/mean_std_total.npy', mean_std_total)
mean_std_total

[(0.01, 0.011109, 0.001602), (0.02, 0.023179, 0.004269), (0.03, 0.036551, 0.0079), (0.04, 0.049895, 0.008882)]
[(0.01, 0.011517, 0.002267), (0.02, 0.022611, 0.003197), (0.03, 0.034158, 0.00583), (0.04, 0.045681, 0.006012)]
[(0.01, 0.011995, 0.002395), (0.02, 0.021421, 0.004373), (0.03, 0.031045, 0.007363), (0.04, 0.040944, 0.010311)]
[(0.01, 0.014554, 0.001798), (0.02, 0.024944, 0.00401), (0.03, 0.034783, 0.006386), (0.04, 0.044452, 0.008614)]


[(0.01, 0.011109, 0.001602),
 (0.02, 0.023179, 0.004269),
 (0.03, 0.036551, 0.0079),
 (0.04, 0.049895, 0.008882),
 (0.01, 0.011517, 0.002267),
 (0.02, 0.022611, 0.003197),
 (0.03, 0.034158, 0.00583),
 (0.04, 0.045681, 0.006012),
 (0.01, 0.011995, 0.002395),
 (0.02, 0.021421, 0.004373),
 (0.03, 0.031045, 0.007363),
 (0.04, 0.040944, 0.010311),
 (0.01, 0.014554, 0.001798),
 (0.02, 0.024944, 0.00401),
 (0.03, 0.034783, 0.006386),
 (0.04, 0.044452, 0.008614)]